In [22]:
pip install -U langchain-openai

Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-1.86.0-py3-none-any.whl (730 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.28.0
    Uninstalling openai-0.28.0:
      Successfully uninstalled openai-0.28.0
Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install openai==0.28

Defaulting to user installation because normal site-packages is not writeable
  Using cached openai-0.28.0-py3-none-any.whl (76 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.86.0
    Uninstalling openai-1.86.0:
      Successfully uninstalled openai-1.86.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
litellm 1.72.0 requires openai>=1.68.2, but you have openai 0.28.0 which is incompatible.
langchain-openai 0.3.23 requires openai<2.0.0,>=1.68.2, but you have openai 0.28.0 which is incompatible.
instructor 1.8.3 requires openai<2.0.0,>=1.70.0, but you have openai 0.28.0 which is incompatible.
crewai 0.130.0 requires openai>=1.13.3, but you have openai 0.28.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [24]:
from sarvamai import SarvamAI
SARVAM_API_KEY = "0140c592-6272-4436-8398-111390895f78"
client = SarvamAI(api_subscription_key=SARVAM_API_KEY)

# PlanSearch generator
def generate_setups(topic, n=5):
    prompt = f"""Generate {n} unique and original joke setups about the topic "{topic}".
Only return the setups, numbered, with no punchlines or explanations. Example format:

1. Why did the penguin wear sunglasses at night?
2. What happens when a penguin tries online dating?

Now generate {n} setups in that format."""
    
    response = client.chat.completions(
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9,
        max_tokens=300
    )
    
    setups_raw = response.choices[0].message.content
    
    # Parse list format (1. ..., 2. ..., etc.)
    setups = [line.strip("1234567890. ").strip() for line in setups_raw.split("\n") if line.strip()]
    return setups[:n]

def generate_punchlines(setup, n=3):
    prompt = f"""Generate {n} punchlines for the following joke setup:

"{setup}"

Only return the punchlines, numbered, with no extra commentary or explanation. Example:

1. Because he was trying to look cool in the moonlight.
2. Because the club only played Arctic Monkeys.

Now generate {n} punchlines in that format."""

    
    response = client.chat.completions(
        messages=[{"role": "user", "content": prompt}],
        temperature=0.9, # increased tempereature for more creative responses.
        max_tokens=300
    )
    
    punchlines_raw = response.choices[0].message.content
    
    punchlines = [line.strip("1234567890. ").strip() for line in punchlines_raw.split("\n") if line.strip()]
    return punchlines[:n]

def generate_jokes(topic="penguins", n_setups=3, n_punchlines=2): # "penguins" just act as a placeholder. The actual input is taken from the user.
    setups = generate_setups(topic, n=n_setups)
    jokes = []

    for setup in setups:
        punchlines = generate_punchlines(setup, n=n_punchlines)
        for punch in punchlines:
            jokes.append(f"{setup} {punch}")

    return jokes

joke_list = []
topic = input("Enter topic for joke generation: ")
jokes = generate_jokes(topic, n_setups=3, n_punchlines=2)
for i, joke in enumerate(jokes, 1):
    print(f"{i}. {joke}")
    joke_list.append(joke)

1. Why did the Node.js process refuse to leave the VM? Because it couldn't handle the async nature of expressing itself
2. Why did the Node.js process refuse to leave the VM? Because it was too deep in the event loop to break free
3. What happens when a Node.js developer gets stuck in a virtual machine? He couldn't escape the event loop
4. What happens when a Node.js developer gets stuck in a virtual machine? He got lost in callback hell
5. How does a Node.js app escape the VM? It breaks out of the event loop
6. How does a Node.js app escape the VM? It calls process.exit() to leave the runtime


In [25]:
def query_openrouter(model, messages):
    body = {
        "model": model,
        "messages": messages,
        "temperature": 0.7
    }
    response = requests.post(BASE_URL, headers=HEADERS, json=body)
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
        return "[ERROR]"
    return response.json()["choices"][0]["message"]["content"]

import re

def extract_final_score(verdict_text):
    match = re.search(r'(\d+)\s+out of\s+10', verdict_text)
    if match:
        return int(match.group(1))
    else:
        return None

In [26]:
import requests
import json
import time

# Set up base config
API_KEY = "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"  # OpenRouter API key
BASE_URL = "https://openrouter.ai/api/v1/chat/completions"

HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Define the joke
#joke = "Why did the penguin bring sunscreen? Because he was going to a chill party."

score_list = []
score_and_joke_dict = {}
score_and_joke_list = []

assert "Authorization" in HEADERS and "Bearer" in HEADERS["Authorization"], "API Key missing from headers"

for joke in joke_list:

    # Judges with different LLMs
    judges = [
        {
            "name": "Judge A",
            "description": "A sarcastic and cynical comic who despises puns.",
            "model": "mistralai/mixtral-8x7b-instruct"
        },
        {
            "name": "Judge B",
            "description": "A surrealist poet who celebrates absurd and chaotic humor.",
            "model": "meta-llama/llama-3-8b-instruct"
        },
        {
            "name": "Judge C",
            "description": "A professional comedy reviewer with 20 years of balanced experience.",
            "model": "anthropic/claude-3-haiku"
        }
    ]

    # Initial system prompt and setup
    conversation = [
        {"role": "system", "content": "You are a panel of comedy judges discussing and evaluating a joke."},
        {"role": "user", "content": f"The joke is:\n\"{joke}\""},
        {"role": "user", "content": "Each judge will rate the joke (1-10), justify their rating in character, and discuss with others."}
    ]

    # Simulate each judge's initial response
    for judge in judges:
        print(f"\n🔹 {judge['name']} is responding...")
        messages = conversation + [
            {"role": "system", "content": f"You are {judge['name']}. {judge['description']}"},
            {"role": "user", "content": f"As {judge['name']}, give your rating and reasoning for the joke."}
        ]
        reply = query_openrouter(judge["model"], messages)
        conversation.append({"role": "assistant", "content": f"{judge['name']}: {reply}"})
        print(f"{judge['name']}:\n{reply}\n{'-'*60}")
        time.sleep(2)  # gentle pause to avoid rate limits

    # Final verdict from Judge C
    print("\n🎯 === FINAL VERDICT ===\n")
    final_prompt = conversation + [
        {
            "role": "system",
            "content": "You are Judge C. Based on the discussion, summarize the panel's opinions and give a final score with reasoning. Final Score:<number>"
        },
        {
            "role": "user",
            "content": "Judge C, please give your final verdict based on the discussion so far."
        }
    ]

    verdict = query_openrouter("anthropic/claude-3-haiku", final_prompt)
    rating = extract_final_score(verdict)
    score_list.append(rating)
    score_and_joke_dict = {
        "joke":joke,
        "score":rating
    }
    score_and_joke_list.append(score_and_joke_dict)
    print(verdict)


🔹 Judge A is responding...
Judge A:
 Judge A: (rolls eyes) Fine, I'll rate the joke. I give it a 3. It's not completely unfunny, but it's just not my type of humor. You see, I'm a comic who appreciates wit and cleverness, and this joke feels a bit too "inside baseball" for me. It's a programming joke, relying on the audience knowing about Node.js, VMs, and asynchronous programming. If you're part of that niche crowd, you might find it amusing, but for the rest of us, it falls flat.

Now, I understand that comedy is subjective, and some people might find this joke hilarious. But for me, it's just not enough. I've heard better puns in a third-grade talent show. I'm not saying the joke is terrible, but it could use a bit more work to make it universally funny.

[To the other judges] What do you think, guys? Am I being too harsh here? Or can we all agree that this joke is a bit... niche?
------------------------------------------------------------

🔹 Judge B is responding...
Judge B:
The 

In [27]:
min = score_list[0]
max = score_list[4]
for i in score_list:
    if i!=None and i<min:
        min = i
    elif i != None and i>max:
        max = i
print(min)
print(max)

6
8


In [28]:
print("Top Joke from the scoring of our respected judges is...\n\n")
for score_and_joke_dicts in score_and_joke_list:
    if (score_and_joke_dicts['score'] == max):
        print(score_and_joke_dicts['joke'])

Top Joke from the scoring of our respected judges is...


Why did the Node.js process refuse to leave the VM? Because it was too deep in the event loop to break free
